In [245]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import seaborn as sns
# connectionString = "dbname='project01' user='pgadmin@fintech-postgres' host='fintech-postgres.postgres.database.azure.com' password='CgYRz!)[4pfB' port='5432' sslmode='true'"
engine = create_engine(
    "postgresql://pgadmin@fintech-postgres:CgYRz!)[4pfB@fintech-postgres.postgres.database.azure.com:5432/project01")
query = """
select portfolio_name, weight, "date" as close_date, 
    close_price, ticker_name, covidineffect 
from portfolio p
    inner join portfolio_tickers pt
    on p.portfolio_id = pt.portfolio_id
    inner join price
    on pt.ticker_id = price.ticker_id
;"""
df = pd.read_sql(query, engine)
df.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect
0,food,0.25,2019-02-19,41.85,ADM,False
1,food,0.25,2019-02-20,41.97,ADM,False
2,food,0.25,2019-02-21,42.57,ADM,False
3,food,0.25,2019-02-22,42.49,ADM,False
4,food,0.25,2019-02-25,42.43,ADM,False


In [246]:
df["porttick"] = df["portfolio_name"] + "-" + df["ticker_name"]
df.head()

,portfolio_name,weight,close_date,close_price,ticker_name,covidineffect,porttick
0,food,0.25,2019-02-19,41.85,ADM,False,food-ADM
1,food,0.25,2019-02-20,41.97,ADM,False,food-ADM
2,food,0.25,2019-02-21,42.57,ADM,False,food-ADM
3,food,0.25,2019-02-22,42.49,ADM,False,food-ADM
4,food,0.25,2019-02-25,42.43,ADM,False,food-ADM


In [247]:
# dfMine = df.loc[df["portfolio_name"]=="hedge"]
# dfMine.head()

In [248]:
# dfCovidNo = dfMine.loc[dfMine["covidineffect"] == False]
# dfCovidNoPivotH = dfCovidNo.pivot(index="close_date", columns='porttick', values="close_price")
# dfCovidNoPivotH.head()

In [249]:
# dfCovid = dfMine.loc[dfMine["covidineffect"] == True]
# dfCovidPivotH = dfCovid.pivot(index="close_date", columns='porttick', values="close_price")
# dfCovidPivotH.head()

In [250]:
query = "select * from portfolio"
dfPortfolios = pd.read_sql(query, engine)
for _, row in dfPortfolios.iterrows():
    if row["portfolio_name"] == "S&P500":
        dfPortfolio = df.loc[df["portfolio_name"]==row["portfolio_name"]]
        dfFiltered = dfPortfolio.loc[dfPortfolio["covidineffect"].values == incovid]
        dfPivot500 = dfFiltered.pivot(index="close_date", columns='ticker_name', values="close_price")
        # Calculate variance of all daily returns o
        variance = dfPivot500['SP500'].var()
        continue 
    for incovid in [True,False]:
        dfPortfolio = df.loc[df["portfolio_name"]==row["portfolio_name"]]
        dfFiltered = dfPortfolio.loc[dfPortfolio["covidineffect"].values == incovid]
        dfPivot = dfFiltered.pivot(index="close_date", columns='ticker_name', values="close_price")
        # Use the `pct_change` function to calculate daily returns      
        daily_returns = dfPivot.pct_change()
        # Use the `mean` function to calculate the mean of daily returns
        avg_daily_return = daily_returns.mean()
        # Use the `std` function to calculate the standard deviation of daily returns
        std_dev_daily_return = daily_returns.std()
        # Use the `std` function and multiply by the square root of the number of trading days in a year to get annualized volatility
        volatility = daily_returns.std() * np.sqrt(252)
        volatility.sort_values(inplace=True)
        # Set weights for corresponding risk profile of stocks, use the `dot` function to multiply each weight by the corresponding stock daily return
        weights = [0.25,0.25,0.25,0.25]
        portfolio_returns = daily_returns.dot(weights)
        # Use the `cumprod` function to calculate cumulative returns
        cumulative_returns = (1 + portfolio_returns).cumprod()
        # Plot the returns of the portfolio in terms of money
        initial_investment = 10000
        cumulative_profit = initial_investment * cumulative_returns
        # cumulative_profit.plot()
        # Use the `corr` function to calculate the correlation between stock returns
        correlation = daily_returns.corr()
        # Use the `heatmap` function from the Seaborn library to visualize the correlation table
        # sns.heatmap(correlation, vmin=-1, vmax=1)
        # Drop highly correlated stocks and keep only the non-correlated stocks
        noncorrelated_daily_returns = daily_returns#.drop(columns=[???])
        # Use the `mean` and `std` functions to calculate the annualized sharpe ratio
        sharpe_ratios = (noncorrelated_daily_returns.mean() * 252) / (noncorrelated_daily_returns.std() * np.sqrt(252))
        # Drop negative sharpe ratio stocks and keep only the postive sharpe ratio stocks
        candidate_daily_returns = noncorrelated_daily_returns#.drop(columns=[???])
        print(candidate_daily_returns.head())
        # Set Portfolio Weights, Calculate Daily and Cumulative Portfolio Returns, and Plot $10,000 Investment Over Time for Optimized Portfolio
        initial_investment = 10000
        weights = [0.25,0.25,0.25,0.25]
        candidate_portfolio_returns = candidate_daily_returns.dot(weights)
        candidate_cumulative_returns = (1 + candidate_portfolio_returns).cumprod()
        candidate_cumulative_profits = (initial_investment * candidate_cumulative_returns)
        # candidate_cumulative_profits.plot(figsize=(20,10))
        # Calculate covariance of all daily returns
        for n in range(0,4): 
            covariance = daily_returns.iloc[:,n].cov(dfPivot500['SP500'])
            beta = covariance / variance
        
      

ticker_name       ADM       CAG       HRL       TSN
close_date                                         
2019-02-19        NaN       NaN       NaN       NaN
2019-02-20   0.002867  0.000833 -0.003926  0.010486
2019-02-21   0.014296  0.001665 -0.026432 -0.008491
2019-02-22  -0.001879 -0.028666  0.021910  0.004123
2019-02-25  -0.001412 -0.002994 -0.008856 -0.015793
ticker_name       ADM       CAG       HRL       TSN
close_date                                         
2019-02-19        NaN       NaN       NaN       NaN
2019-02-20   0.002867  0.000833 -0.003926  0.010486
2019-02-21   0.014296  0.001665 -0.026432 -0.008491
2019-02-22  -0.001879 -0.028666  0.021910  0.004123
2019-02-25  -0.001412 -0.002994 -0.008856 -0.015793
ticker_name      CHTR     CMCSA         T        VZ
close_date                                         
2019-02-19        NaN       NaN       NaN       NaN
2019-02-20   0.003817  0.006391  0.007183  0.005772
2019-02-21  -0.000629  0.017994 -0.000648  0.006994
2019-02-22  

<function print>

In [251]:
correlation = daily_returns.corr()
correlation.head()

ticker_name,ABT,JNJ,MRK,PFE
ticker_name,,,,
ABT,1.000000,0.330805,0.562597,0.396142
JNJ,0.330805,1.000000,0.280747,0.238815
MRK,0.562597,0.280747,1.000000,0.408607
PFE,0.396142,0.238815,0.408607,1.000000
